In [49]:
import keras.backend as K
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

from keras.models import load_model
from keras.layers import Input
from yolo3.utils import letterbox_image

from yolo3.model import yolo_eval, yolo_body, tiny_yolo_body, yolo_body_with_classification, yolo_backbone

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"


K.clear_session()
'''
model0 = yolo_body(Input(shape=(None,None,3)), 9//3, 10)
model0.load_weights("baseline.h5",by_name=True) # make sure model, anchors and classes match

#print(model.summary())
'''
model1 = yolo_body_with_classification(Input(shape=(None,None,3)), 9//3, 10)
model1.load_weights("yolo_with_classification.h5",by_name=True) # make sure model, anchors and classes match

print(model1.summary())

C:\Users\pcl\Desktop\keras-yolo3-master-pcltest\yolo3\model.py:110: UserWarning: Update your `GlobalAveragePooling2D` call to the Keras 2 API: `GlobalAveragePooling2D(data_format="channels_last")`
  classification_results = GlobalAveragePooling2D(dim_ordering='channels_last')(darknet.output)


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, None, None, 3 128         conv2d_1[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)       (None, None, None, 3 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
zero_paddi

In [64]:
[0,0,0] + [1,0,1]

[0, 0, 0, 1, 0, 1]

In [50]:
import cv2
from yolo3.utils import letterbox_image
from PIL import Image, ImageFont, ImageDraw

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

img_dir = 'E:\\VOC2019\\VOC2019\\JPEGImages\\'
annotation_list = 'E:/VOC2019/VOC2019/annotation_list.txt'
cls=["normal","ascus","asch","lsil","hsil","agc","adenocarcinoma","vaginalis","monilia","dysbacteriosis"]
detectiondir = 'C:/Users/pcl/Desktop/fucking test/detections_baseline/'
#models=[model0,model1]
model = model1

with open(annotation_list) as f:
    lines = f.readlines()
    l = len(lines)
    for line in lines:
        img = line.split()[0]
        img_id = img.rstrip('.jpg')
        boxes = line.split()[1:]
        img_path = img_dir + img
        
        
        #img = image.load_img(img_path, target_size=(416, 416))   # 大小为416*416的Python图像库图像
        #x= image.img_to_array(img)  # 形状为（416， 416， 3）的float32格式Numpy数组
        x = Image.open(img_path)
        boxed_image = letterbox_image(x, (416,416))
        x = np.array(boxed_image, dtype='float32')
        x /= 255.0
        x = np.expand_dims(x, axis=0)  # 添加一个维度，将数组转化为（1， 416， 416， 3）的形状批量
        
        #x = preprocess_input(x)   #按批量进行预处理（按通道颜色进行标准化）

        for index in [0]:
            #model = models[index]
            preds = model.predict(x)

            #layername = "conv2d_"+str(52+75*index)
            #layername = "add_"+str(23+index*23)
            layername = "conv2d_58"
            last_conv_layer = model.get_layer(layername)  # add_23层的输出特征图

            grads = K.gradients(model.outputs[1][0][:,:,5], last_conv_layer.output)[0]   # 类别相对于add_23输出特征图的梯度

            pooled_grads = K.mean(grads, axis=(0, 1, 2))   # 形状是（512， ）的向量，每个元素是特定特征图通道的梯度平均大小

            iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])  # 这个函数允许我们获取刚刚定义量的值：对于给定样本图像，pooled_grads和block5_conv3层的输出特征图

            pooled_grads_value, conv_layer_output_value = iterate([x])  # 给我们两个大象样本图像，这两个量都是Numpy数组

            for i in range(1024):
                conv_layer_output_value[:, :, i] *= pooled_grads_value[i]  # 将特征图数组的每个通道乘以这个通道对大象类别重要程度

            heatmap = np.mean(conv_layer_output_value, axis=-1)  # 得到的特征图的逐通道的平均值即为类激活的热力图

            heatmap = np.maximum(heatmap, 0)
            heatmap /= np.max(heatmap)

            img = cv2.imread(img_path)  # 用cv2加载原始图像

            heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))  # 将热力图的大小调整为与原始图像相同


            heatmap = np.uint8(255 * (0.8*heatmap))  # 将热力图转换为RGB格式

            heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)   # 将热力图应用于原始图像

            superimposed_img = np.uint8(heatmap) # 这里的0.4是热力图强度因子

            #cv2.imshow('temp', superimposed_img)
            cv2.imwrite("E:\\heatmap02_det\\" + img_id + "_heatmap"+str(index)+".jpg", superimposed_img, [int(cv2.IMWRITE_JPEG_QUALITY),70])   # 将图像保存到硬盘

            #cv2.waitKey(0)


        img = cv2.imread(img_path)
        detectionfile = detectiondir + img_id + '.txt'
        with open(detectionfile) as fd:
            detections = fd.readlines()
        for detection in detections:
            det = detection.split()
            if float(det[1]) < 0.1: continue
            cv2.rectangle(img,(int(float(det[2])) , int(float(det[3]))) , (int(float(det[4])), int(float(det[5]))), (0,0,255),thickness=2)
            cv2.putText(img,det[0] + ' ' + det[1],(int((float(det[2]))) - 5,int(float(det[3])) - 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), thickness=2)
        img = cv2.resize(img,(960,720),interpolation=cv2.INTER_AREA)    
        cv2.imwrite("E:\\heatmap02_det\\" + img_id + "_det.jpg", img, [int(cv2.IMWRITE_JPEG_QUALITY),70])    

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: RuntimeWarning: invalid value encountered in true_divide


KeyboardInterrupt: 

In [9]:

def visualize_bbox(imgpath, boxes, detectionfile, show_thresh=0.1):
    cls=["normal","ascus","asch","lsil","hsil","agc","adenocarcinoma","vaginalis","monilia","dysbacteriosis"]
    
    img = cv2.imread(imgpath)
    with open(detectionfile) as fd:
        detections = fd.readlines()
    
    for boxline in boxes:
        box = boxline.split(',')
        print(box[0:])
        #cv2.rectangle(img,(int(float(box[0])) , int(float(box[1]))) , (int(float(box[2])), int(float(box[3]))), (0,255,0),thickness=2)
        #cv2.putText(img,cls[int(box[4])],(int(float(box[0])),int(float(box[1]))),cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255,0), thickness=2)
    
    
    for detection in detections:
        det = detection.split()
        if float(det[1]) < show_thresh: continue
        cv2.rectangle(img,(int(float(det[2])) , int(float(det[3]))) , (int(float(det[4])), int(float(det[5]))), (0,0,255),thickness=2)
        cv2.putText(img,det[0] + ' ' + det[1],(int((float(det[2]))) - 5,int(float(det[3])) - 5),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), thickness=2)
    
    img = cv2.resize(img,(960,720),interpolation=cv2.INTER_AREA)
    cv2.imshow(imgpath,img)
    cv2.imwrite('temp.png', img, [int(cv2.IMWRITE_JPEG_QUALITY),70])
    cv2.waitKey(0)


In [8]:
# ht07612_20.jpg
# ht32378_06.jpg 
# ht69457_18.jpg 
# ht89453_10.jpg
# ht10212_21.jpg
str(1)

'1'